In [63]:
using FFTW
using Statistics
using TyPlot
using TyStatistics


# 数据设置
exdata = [
    9007  7750  8162  7717  7792  7836
    8106  6981  7306  7461  6957  6892
    8928  8038  8124  7776  7726  7791
    9137  8422  7870  7925  8106  8129
   10017  8714  9387  8634  8890  9115
   10826  9512  9556  8945  9299  9434
   11317 10120 10093 10078 10625 10484
   10744  9823  9620  9179  9302  9827
    9713  8743  8285  8037  8314  9110
    9938  9129  8433  8488  8850  9070
    9161  8710  8160  7874  8265  8633
    8927  8680  8034  8647  8796  9240
]
ts = reshape(exdata, 72, 1)
years = LinRange(1973, 1979, 72)

# 绘制原始数据
plot(years, ts, marker="o")
xlabel("Year")
#ylabel("Number of Accidental Deaths")

# 计算并绘制傅里叶变换的振幅
tsdft = fft(ts .- mean(ts))
freq = 0:1/72:1/2
figure(2)
plot(freq .* 12, abs.(tsdft[1:length(tsdft)÷2+1]), marker = "o")
xlabel("Cycles/Year")
#ylabel("Magnitude") 

# 使用频率域进行拟合
freqbin = 72 ÷ 12
freqbins = [freqbin, 72-freqbin] .+ 1
tsfit = zeros(Complex{Float64}, 72)
tsfit[freqbins] = tsdft[freqbins]
tsfit = ifft(tsfit)
mu = mean(ts)
tsfit_real = real(mu .+ tsfit)

# 绘制数据和拟合模型
figure(3)
plot(years, ts, marker ="o",label = "Data")
xlabel("Year") 
#ylabel("Number of Accidental Deaths")
hold("on")
plot(years, tsfit_real, linewidth = 2, label = "Fitted Model")

# 计算并绘制残差的自相关
resid = ts .- tsfit_real
xc, lags = xcorr(resid, 50, "coeff")
lconf_val = -1.96 / sqrt(72)
uconf_val = 1.96 / sqrt(72)
lconf = fill(lconf_val, length(xc[51:end]))
uconf = fill(uconf_val, length(xc[51:end]))
figure(4)
stem(lags[51:end], xc[51:end], label = "Autocorrelation")
hold("on")
plot(lags[51:end], lconf)
plot(lags[51:end], uconf)


1-element Vector{PyCall.PyObject}:
 PyObject <matplotlib.lines.Line2D object at 0x00000219A26A79C8>

In [64]:
using TyMath
tsfit2dft = zeros(Complex{Float64}, 72)
Y = sort(abs.(tsdft), rev=true, dims=1)
I = sortperm(abs.(tsdft), rev=true, dims=1)
indices = I[1:6]
tsfit2dft[indices] .= tsdft[indices]

norm1 = norm(tsdft / sqrt(72), 2) / norm(ts .- mean(ts), 2)
norm2 = norm(tsfit2dft / sqrt(72), 2) / norm(ts .- mean(ts), 2)
tsfit2 = mu .+ real(ifft(tsfit2dft))

# Assume `years` is given
# Plot the original data and the fits
figure(5)
plot(years, ts, marker="o", label="Data", )
xlabel("Year")
ylabel("Number of Accidental Deaths")
hold("on")
plot(years, tsfit_real, linewidth = 2, label="1 Frequency")
plot(years, tsfit2, linewidth = 2, label="3 Frequencies")
legend()



PyObject <matplotlib.legend.Legend object at 0x00000219A3E746C8>

In [65]:
using Statistics

# 假设 ts 和 tsfit2 是给定的时间序列数据

# 计算残差
resid = ts .- tsfit2

# 计算自相关系数
xc, lags = xcorr(resid, 50, "coeff")
figure(6)
hold("on")
# 绘制自相关系数图像
stem(lags[51:end], xc[51:end], filled=true)
xlabel("Lag")
ylabel("Correlation Coefficient")
title("Autocorrelation of Residuals")

# 绘制置信区间
lconf = -1.96 * ones(51) / sqrt(72)
uconf = 1.96 * ones(51) / sqrt(72)
plot(lags[51:end], lconf)
plot(lags[51:end], uconf)



true